## 例とアプリケーション

このレッスンでは、いくつかの変分アルゴリズムの例とその適用方法について説明します。

- カスタム変分アルゴリズムの書き方
- 変分アルゴリズムを適用して最小固有値を見つける方法
- 変分アルゴリズムを利用してアプリケーションのユースケースを解決する方法

### 問題の定義

変分アルゴリズムを使用して、次の観測可能量の固有値を見つけたいと想像してください。

$$
\hat{O}_1 = 2 II - 2 XX + 3 YY - 3 ZZ,
$$

この観測可能量は次の固有値を持ちます。

$$
\left\{
\begin{array}{c}
\lambda_0 = -6 \\
\lambda_1 = 4 \\
\lambda_2 = 4 \\
\lambda_3 = 6
\end{array}
\right\}
$$

そして固有状態：

$$
\left\{
\begin{array}{c}
|\phi_0\rangle = \frac{1}{\sqrt{2}}(|00\rangle + |11\rangle)\\
|\phi_1\rangle = \frac{1}{\sqrt{2}}(|00\rangle - |11\rangle)\\
|\phi_2\rangle = \frac{1}{\sqrt{2}}(|01\rangle - |10\rangle)\\
|\phi_3\rangle = \frac{1}{\sqrt{2}}(|01\rangle + |10\rangle)
\end{array}
\right\}
$$

In [ ]:
from qiskit.quantum_info import SparsePauliOp

observable = SparsePauliOp.from_list([("II", 2), ("XX", -2), ("YY", 3), ("ZZ", -3)])

## カスタム VQE

最初に、VQE インスタンスを手動で構築して $\hat{O}_1$ の最小固有値を見つける方法を調べます。これには、このコースで説明したさまざまなテクニックが組み込まれています。 

In [ ]:
from qiskit.circuit.library import TwoLocal
from qiskit.quantum_info import SparsePauliOp
from qiskit import QuantumCircuit
from qiskit_ibm_runtime import QiskitRuntimeService, Estimator
import numpy as np

# Add your token below
service = QiskitRuntimeService(
    channel="ibm_quantum",
)

def cost_function_vqe(theta):
    observable = SparsePauliOp.from_list([("II", 2), ("XX", -2), ("YY", 3), ("ZZ", -3)])
    reference_circuit = QuantumCircuit(2)
    reference_circuit.x(0)

    variational_form = TwoLocal(
        2,
        rotation_blocks=["rz", "ry"],
        entanglement_blocks="cx",
        entanglement="linear",
        reps=1,
    )
    ansatz = reference_circuit.compose(variational_form)

    backend = service.backend("ibmq_qasm_simulator")
    
    # Use estimator to get the expected values corresponding to each ansatz
    estimator = Estimator(session=backend)
    job = estimator.run(ansatz, observable, theta)
    values = job.result().values

    return values

このコスト関数を使用して、最適なパラメーターを計算できます

In [ ]:
from qiskit.algorithms.optimizers import COBYLA

initial_theta = np.ones(8)
optimizer = COBYLA()

optimizer_result = optimizer.minimize(fun=cost_function_vqe, x0=initial_theta)

optimal_parameters = optimizer_result.x
print(optimal_parameters)

最後に、最適パラメーターを使用して最小固有値を計算できます。

In [ ]:
observable = SparsePauliOp.from_list([("II", 2), ("XX", -2), ("YY", 3), ("ZZ", -3)])
reference_circuit = QuantumCircuit(2)
reference_circuit.x(0)

variational_form = TwoLocal(
    2,
    rotation_blocks=["rz", "ry"],
    entanglement_blocks="cx",
    entanglement="linear",
    reps=1,
)
ansatz = reference_circuit.compose(variational_form)
solution = ansatz.bind_parameters(optimal_parameters)

backend = service.backend("ibmq_qasm_simulator")
estimator = Estimator(session=backend)
job = estimator.run(solution, observable)
values = job.result().values

experimental_min_eigenvalue = values[0]
print(experimental_min_eigenvalue)

In [ ]:
from numpy.linalg import eigvalsh

solution_eigenvalue = min(eigvalsh(observable.to_matrix()))
print(
    f"Percent error: {abs((experimental_min_eigenvalue - solution_eigenvalue)/solution_eigenvalue):.2e}"
)

ご覧のとおり、結果は理想に非常に近いものになっています。 

## Qiskit の VQE の構成

便宜上、既存の Qiskit [VQE](https://qiskit.org/documentation/stubs/qiskit.algorithms.minimum_eigensolvers.VQE.html)の実装を使うことができて、最初の観測可能量 $\hat{O}_1$ の最小固有値を見つけ、すべての出力結果を調べることもできます。

この場合、以下を使用します。

- 参照演算子 $\equiv I$ の調査を開始して、これにより速度が向上することを示します。
- Qiskit Terra の[`Estimator`](https://qiskit.org/documentation/stubs/qiskit.primitives.Estimator.html#qiskit.primitives.Estimator)
- [`SLSQP`](https://qiskit.org/documentation/stubs/qiskit.algorithms.optimizers.SLSQP.html) (つまり、逐次最小二乗法プログラミング) オプティマイザー
- さらに、 `SLSQP`オプティマイザーの初期ポイントを $\vec\theta_0 = (1, \cdots, 1)$ に設定します。

In [ ]:
from qiskit.primitives import Estimator
from qiskit.algorithms.optimizers import SLSQP
from qiskit.algorithms.minimum_eigensolvers import VQE
import numpy as np

estimator = Estimator()
optimizer = SLSQP()
ansatz = TwoLocal(
    2,
    rotation_blocks=["rz", "ry"],
    entanglement_blocks="cx",
    entanglement="linear",
    reps=1,
)

vqe = VQE(estimator, ansatz, optimizer, initial_point=np.ones(8))

`VQE`インスタンスを初期化したので、 [`VQE.compute_minimum_eigenvalue`](https://qiskit.org/documentation/stubs/qiskit.algorithms.minimum_eigensolvers.VQE.compute_minimum_eigenvalue.html#qiskit.algorithms.minimum_eigensolvers.VQE.compute_minimum_eigenvalue)メソッドで結果を取得できます。結果を見てみましょう。

In [ ]:
result = vqe.compute_minimum_eigenvalue(observable)
print(result)

`optimizer_result`を見てみましょう:

In [ ]:
print(result.optimizer_result)

ただし、これらすべての情報の中で最も重要な部分は固有値です。理論値と比較してみましょう。

In [ ]:
from numpy.linalg import eigvalsh

eigenvalues = eigvalsh(observable.to_matrix())
min_eigenvalue = eigenvalues[0]

print("EIGENVALUES:")
print(f"  - Theoretical: {min_eigenvalue}.")
print(f"  - VQE: {result.eigenvalue}")
print(
    f"Percent error >> {abs((result.eigenvalue - min_eigenvalue)/min_eigenvalue):.2e}"
)

ご覧のとおり、結果は理想に非常に近いものになっています。

ただし、固有状態は`results`の一部ではなかったため、まだ見ていません。この目的のために、最適なパラメーター値`result.optimal_parameters` を `results.optimal_circuit`にバインドし、そのバインドされた (つまり、パラメーター化されていない) 回路から[`Statevector`](https://qiskit.org/documentation/stubs/qiskit.quantum_info.Statevector.html)を定義します。

In [ ]:
from qiskit.quantum_info import Statevector

optimal_circuit = result.optimal_circuit.bind_parameters(result.optimal_parameters)
optimal_vector = Statevector(optimal_circuit)

rounded_optimal_vector = np.round(optimal_vector.data, 3)
print(f"EIGENSTATE: {rounded_optimal_vector}")

この結果は、 $\frac{1}{\sqrt{2}}(|00\rangle + |11\rangle) \equiv [\frac{1}{\sqrt{2} },0,0,\frac{1}{\sqrt{2}}]$ の理論上のものにあまり近くないようです。ただし、固有ベクトルは定数倍まで定義されていることに注意してください。さらに、量子状態は常に正規化されてグローバル位相まで等価であるため、これら 2 つの状態ベクトルが等価であることを簡単に確認できます。

In [ ]:
from numpy.linalg import eigh

_, eigenvectors = eigh(observable.to_matrix())
min_eigenvector = eigenvectors.T[0]  # Note: transpose to extract by index

optimal_vector.equiv(min_eigenvector, atol=1e-4)

得られた状態は、 $10^{-4}$ までは理想的な状態と同等であると結論付けることができます。

### 参照状態を追加

前の例では、参照演算子 $U_R$ を使用していません。ここで、理想的な固有状態 $\frac{1}{\sqrt{2}}(|00\rangle + |11\rangle)$ がどのように得られるかを考えてみましょう。次の回路を考えてみましょう。

In [ ]:
from qiskit import QuantumCircuit

ideal_qc = QuantumCircuit(2)
ideal_qc.h(0)
ideal_qc.cx(0, 1)

ideal_qc.draw("mpl")

この回路が目的の状態になることをすぐに確認できます。

In [ ]:
Statevector(ideal_qc)

解の状態を準備する回路がどのようになるかを見てきたので、アダマール ゲートを参照回路として使用するのが合理的であるように思われるため、完全な ansatz は次のようになります。

In [ ]:
reference = QuantumCircuit(2)
reference.h(0)
# Include barrier to separate reference from variational form
reference.barrier()

ref_ansatz = ansatz.decompose().compose(reference, front=True)

ref_ansatz.draw("mpl")

この新しい回路では、すべてのパラメーターを $0$ に設定して理想的な解に到達できるため、参照回路の選択が妥当であることが確認されます。

ここで、コスト関数の評価、オプティマイザーの反復、および所要時間を前回の試行と比較してみましょう。

In [ ]:
num_evaluations = result.cost_function_evals
num_iterations = result.optimizer_result.nit
time = result.optimizer_time

print("NO REFERENCE STATE:")
print(f"  - Number of evaluations: {num_evaluations}")
print(f"  - Number of iterations: {num_iterations}")
print(f"  - Time: {time:.5f} seconds")

In [ ]:
# You can change the ansatz of the already defined vqe object instead of creating a new one
vqe.ansatz = ref_ansatz

ref_result = vqe.compute_minimum_eigenvalue(observable)

In [ ]:
num_evaluations_ref = ref_result.cost_function_evals
num_iterations_ref = ref_result.optimizer_result.nit
time_ref = ref_result.optimizer_time

print("ADDED REFERENCE STATE:")
print(f"  - Number of evaluations: {num_evaluations_ref}")
print(f"  - Number of iterations: {num_iterations_ref}")
print(f"  - Time: {time_ref:.5f} seconds")
print()

if num_evaluations_ref < num_evaluations:
    print(">> Number of cost function evaluations improved")
elif num_evaluations_ref > num_evaluations:
    print(">> Number of cost function evaluations worsened")
if num_iterations_ref < num_iterations:
    print(">> Number of iterations improved")
elif num_iterations_ref > num_iterations:
    print(">> Number of iterations worsened")
if time_ref < time:
    print(">> Time improved")
elif time_ref > time:
    print(">> Time worsened")

### 初期点の変更

参照状態を追加する効果を見たので、異なる初期点 $\vec{\theta_0}$ を選択するとどうなるかを見ていきます。特に、 $\vec{\theta_0}=(0,0,0,0,6,0,0,0)$ と $\vec{\theta_0}=(6,6,6,6,6,6,6,6,6)$を使用します。参照状態が導入されたときに説明したように、すべてのパラメーターが $0$ のときに理想的な解が見つかることを思い出してください。したがって、最初の初期点では、評価、反復、および時間が少なくて済みます。

In [ ]:
vqe.initial_point = [0, 0, 0, 0, 6, 0, 0, 0]

result = vqe.compute_minimum_eigenvalue(observable)

num_evaluations = result.cost_function_evals
num_iterations = result.optimizer_result.nit
time = result.optimizer_time

print(f"INITIAL POINT: {vqe.initial_point}")
print(f"  - Number of evaluations: {num_evaluations}")
print(f"  - Number of iterations: {num_iterations}")
print(f"  - Time: {time:.5f} seconds")

In [ ]:
vqe.initial_point = 6 * np.ones(8)

result = vqe.compute_minimum_eigenvalue(observable)

num_evaluations = result.cost_function_evals
num_iterations = result.optimizer_result.nit
time = result.optimizer_time

print(f"INITIAL POINT: {vqe.initial_point}")
print(f"  - Number of evaluations: {num_evaluations}")
print(f"  - Number of iterations: {num_iterations}")
print(f"  - Time: {time:.5f} seconds")

## VQD の例

ここで、観測可能量の最小固有値だけを探す代わりに、すべての $4$ を探します。前の章の表記 (および Qiskit の[VQD](https://qiskit.org/documentation/stubs/qiskit.algorithms.eigensolvers.VQD.html)クラスの表記) に従うと、これは $k=4$ を意味します。

VQD のコスト関数は次のとおりです。

$$
C_{l}(\vec{\theta}) := 
\langle \psi(\vec{\theta}) | \hat{H} | \psi(\vec{\theta})\rangle + 
\sum_{j=0}^{l-1}\beta_j |\langle \psi(\vec{\theta})| \psi(\vec{\theta^j})\rangle  |^2 
\quad \forall l\in\{1,\cdots,k\}=\{1,\cdots,4\}
$$

これは、 `VQD` オブジェクトを定義するときに、ベクトル$\vec{\beta}=(\beta_0,\cdots,\beta_{k-1})$ (この場合は $(\beta_0, \beta_1, \beta_2, \beta_3)$)を引数として渡さなければならないので、特に重要です。

また、QiskitのVQDの実装では、前のノートブックで説明した実効的な観測可能量を考慮する代わりに、 [`ComputeUncompute`](https://qiskit.org/documentation/stubs/qiskit.algorithms.state_fidelities.ComputeUncompute.html) アルゴリズムによって、忠実度 $|\langle \psi(\vec{\theta})| \psi(\vec{\theta^j})\rangle  |^2$ を直接計算しています。これは `Sampler` Primitiveを使って、回路 $U_A^\dagger(\vec{\theta})U_A(\vec{\theta^j})$ に対して $|0\rangle$ を得る確率をサンプリングするものです。この確率は以下のとおりですから、このことは厳密に機能しています。

$$
\begin{aligned}

p_0

&amp; = |\langle 0|U_A^\dagger(\vec{\theta})U_A(\vec{\theta^j})|0\rangle|^2 \\[1mm]

&amp; = |\big(\langle 0|U_A^\dagger(\vec{\theta})\big)\big(U_A(\vec{\theta^j})|0\rangle\big)|^2 \\[1mm]

&amp; = |\langle \psi(\vec{\theta}) |\psi(\vec{\theta^j}) \rangle|^2 \\[1mm]

\end{aligned}
$$

最後に、新しいオプティマイザーを試すために、 <code>SLSQP</code>の代わりに<a><code data-md-type="codespan">COBYLA</code></a>を使用してみましょう。

In [ ]:
from qiskit.primitives import Sampler
from qiskit.algorithms.optimizers import COBYLA
from qiskit.algorithms.state_fidelities import ComputeUncompute
from qiskit.algorithms.eigensolvers import VQD

optimizer = COBYLA()
sampler = Sampler()
estimator = Estimator()
fidelity = ComputeUncompute(sampler)

k = 4
betas = [40, 60, 30, 30]

vqd = VQD(estimator, fidelity, ansatz, optimizer, k=k, betas=betas)

前の例との唯一の API の違いとして、`VQE.compute_minimum_eigenvalue`メソッドを呼び出す代わりに、 [`VQD.compute_eigenvalues`](https://qiskit.org/documentation/stubs/qiskit.algorithms.eigensolvers.VQD.compute_eigenvalues.html)を呼び出すことに注意してください。次の観測可能量を調べることから始めましょう。

$$
\hat{O}_2 := 2 II - 3 XX + 2 YY - 4 ZZ
$$

この観測可能量は次の固有値を持ちます。

$$
\left\{
\begin{array}{c}
\lambda_0 = -7 \\
\lambda_1 = 3\\
\lambda_2 = 5 \\
\lambda_3 = 7
\end{array}
\right\}
$$

そして固有状態：

$$
\left\{
\begin{array}{c}
|\phi_0\rangle = \frac{1}{\sqrt{2}}(|00\rangle + |11\rangle)\\
|\phi_1\rangle = \frac{1}{\sqrt{2}}(|00\rangle - |11\rangle)\\
|\phi_2\rangle = \frac{1}{\sqrt{2}}(|01\rangle + |10\rangle)\\
|\phi_3\rangle = \frac{1}{\sqrt{2}}(|01\rangle - |10\rangle)
\end{array}
\right\}
$$

In [ ]:
observable_2 = SparsePauliOp.from_list([("II", 2), ("XX", -3), ("YY", 2), ("ZZ", -4)])

result = vqd.compute_eigenvalues(observable_2)
print(result)

取得した[`VQDResult`](https://qiskit.org/documentation/stubs/qiskit.algorithms.eigensolvers.VQDResult.html) は、 `VQEResult`と完全に類似しています。各属性が $i$ 番目の要素が $i$ 番目の固有値に対応するリストであるという点だけが異なります。

さて、固有値を見たところで、実験の固有ベクトルと理論の固有ベクトルを比較してみましょう：

In [ ]:
optimal_circuits = [
    circuit.bind_parameters(parameters)
    for circuit, parameters in zip(result.optimal_circuits, result.optimal_parameters)
]
eigenstates = [Statevector(c) for c in optimal_circuits]

for i, (eigenvalue, eigenstate) in enumerate(zip(result.eigenvalues, eigenstates)):
    eigenvalue = eigenvalue.real
    eigenstate = np.round(eigenstate.data, 3).tolist()
    print(f"RESULT {i}:")
    print(f"  - {eigenvalue = :.3f}")
    print(f"  - {eigenstate = }")

これらの結果は、小さな近似誤差とグローバル位相を除いて、予想される結果と同じです。

ここで、最初に観測可能量 $\hat{O}_1 := 2 II - 2 XX + 3 YY - 3 ZZ$ についてこの問題を解いてみましょう。

In [ ]:
result = vqd.compute_eigenvalues(observable)

optimal_circuits = [
    circuit.bind_parameters(parameters)
    for circuit, parameters in zip(result.optimal_circuits, result.optimal_parameters)
]
eigenstates = [Statevector(c) for c in optimal_circuits]

for i, (eigenvalue, eigenstate) in enumerate(zip(result.eigenvalues, eigenstates)):
    eigenvalue = eigenvalue.real
    eigenstate = np.round(eigenstate.data, 3).tolist()
    print(f"RESULT {i}:")
    print(f"  - {eigenvalue = :.3f}")
    print(f"  - {eigenstate = }")

ここで、 $\lambda_1 = \lambda_2 = 4$ に対応する固有状態は、予想されるものと同じではありません:

$$
|\phi_1\rangle = \frac{1}{\sqrt{2}}(|00\rangle - |11\rangle) \\
|\phi_2\rangle = \frac{1}{\sqrt{2}}(|01\rangle + |10\rangle)
$$

これは、 $\lambda_1=\lambda_2$ のように、複雑な線形結合も同じ固有値を持つ固有状態であるためです。

$$
\begin{aligned}
\alpha_1 |\phi_1\rangle + \alpha_2 |\phi_2\rangle
& = \frac{1}{\sqrt{2}}(\alpha_1 |00\rangle + \alpha_2 |01\rangle - \alpha_2 |10\rangle - \alpha_1 |11\rangle) \\[1mm]
& \equiv \frac{1}{\sqrt{2}}[\alpha_1, \alpha_2, -\alpha_2, -\alpha_1]
\end{aligned}
$$

それはまさに、これらの結果で私たちが見ているものです。

### ベータの変更

インスタンスのレッスンで述べたように、 $\vec{\beta}$ の値は固有値の差より大きくなければなりません。 $\hat{O}_2$ でその条件を満たさない場合に何が起こるか見てみましょう

$$
\hat{O}_2 = 2 II - 3 XX + 2 YY - 4 ZZ
$$

固有値は

$$
\left\{
\begin{array}{c}
\lambda_0 = -7 \\
\lambda_1 = 3\\
\lambda_2 = 5 \\
\lambda_3 = 7
\end{array}
\right\}
$$

In [ ]:
vqd.betas = np.ones(4)
result = vqd.compute_eigenvalues(observable_2)

optimal_circuits = [
    circuit.bind_parameters(parameters)
    for circuit, parameters in zip(result.optimal_circuits, result.optimal_parameters)
]
eigenstates = [Statevector(c) for c in optimal_circuits]

for i, (eigenvalue, eigenstate) in enumerate(zip(result.eigenvalues, eigenstates)):
    eigenvalue = eigenvalue.real
    eigenstate = np.round(eigenstate.data, 3).tolist()
    print(f"RESULT {i}:")
    print(f"  - {eigenvalue = :.3f}")
    print(f"  - {eigenstate = }")

今回、オプティマイザーは、すべての固有状態に対する提案された解として、同じ状態 $|\phi_0\rangle = \frac{1}{\sqrt{2}}(|00\rangle + |11\rangle)$ を返しますがこれは明らかに誤っています。これは、ベータが小さすぎて、連続するコスト関数の最小固有状態にペナルティを課すことができないために発生します。したがって、アルゴリズムの後の反復で有効な検索スペースから除外されず、可能な最良解として常に選択されるのです。

$\vec{\beta}$ の値を試して、固有値の差よりも大きいことを確認することをお勧めします。 

## 量子化学: 基底状態と励起エネルギーソルバー

私たちの目的は、観測可能量を表わすエネルギー (ハミルトニアン $\hat{\mathcal{H}}$) の期待値を最小化することです。

$$
\min_{\vec\theta} \langle\psi(\vec\theta)|\hat{\mathcal{H}}|\psi(\vec\theta)\rangle
$$

In [ ]:
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.circuit.library import HartreeFock, UCC
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit.quantum_info import SparsePauliOp

H2_op = SparsePauliOp.from_list(
    [
        ("II", -1.052373245772859),
        ("IZ", 0.39793742484318045),
        ("ZI", -0.39793742484318045),
        ("ZZ", -0.01128010425623538),
        ("XX", 0.18093119978423156),
    ]
)

driver = PySCFDriver(
    atom="H 0 0 0; H 0 0 0.735",
    basis="sto3g",
    charge=0,
    spin=0,
    unit=DistanceUnit.ANGSTROM,
)

h2_problem = driver.run()

mapper = JordanWignerMapper()

h2_reference_state = HartreeFock(
    num_spatial_orbitals=h2_problem.num_spatial_orbitals,
    num_particles=h2_problem.num_particles,
    qubit_mapper=mapper,
)

ansatz = UCC(
    num_spatial_orbitals=h2_problem.num_spatial_orbitals,
    num_particles=h2_problem.num_particles,
    qubit_mapper=mapper,
    initial_state=h2_reference_state,
    excitations=2,
)

ansatz.decompose().decompose().draw("mpl")

In [ ]:
from qiskit.algorithms.minimum_eigensolvers import VQE
from qiskit.primitives import Sampler, Estimator
from qiskit.algorithms.state_fidelities import ComputeUncompute
from qiskit_nature.second_q.algorithms import GroundStateEigensolver

estimator = Estimator()
sampler = Sampler()
fidelity = ComputeUncompute(sampler)

vqe = VQE(
    estimator=estimator,
    ansatz=ansatz,
    optimizer=optimizer,
    initial_point=np.zeros(ansatz.num_parameters),
)
gse = GroundStateEigensolver(qubit_mapper=mapper, solver=vqe)
result = gse.solve(h2_problem)

print(result)

VQD を活用して励起状態を解くこともできます

In [ ]:
from qiskit.algorithms.eigensolvers import VQD

h2_operators, _ = gse.get_qubit_operators(h2_problem)

optimizer = COBYLA()
sampler = Sampler()
estimator = Estimator()
fidelity = ComputeUncompute(sampler)

k = 3
betas = [33, 33, 33]

vqd = VQD(
    estimator=estimator,
    ansatz=ansatz,
    optimizer=optimizer,
    fidelity=fidelity,
    initial_point=np.zeros(ansatz.num_parameters),
    k=k,
    betas=betas,
)
result = vqd.compute_eigenvalues(operator=h2_operators)
vqd_values = result.optimal_values
print(vqd_values)

## 最適化: マックスカット

マックスカット（Max-Cut）問題とは、グラフの頂点を2つの互いに素な集合に分割し、その2つの集合間の辺の数が最大になるようにする組合せ最適化問題です。より正式には、 $V$ が頂点の集合、 $E$ が辺の集合である無向グラフ $G=(V,E)$ が与えられたとき、マックスカット問題は、頂点を2つの互いに素な部分集合 $S$ と$T$に分割する際に、一方の端点が $S$ にあり他方が $T$ にあるような辺の数が最大となるような分割を求めます。

マックスカットを応用して、クラスタリング、ネットワーク設計、相転移など様々な問題を解くことができます。まず、問題グラフを作成することから始めます：

In [ ]:
import networkx as nx

n = 4
G = nx.Graph()
G.add_nodes_from(range(n))
edge_list = [(0, 1, 1.0), (0, 2, 1.0), (0, 3, 1.0), (1, 2, 1.0), (2, 3, 1.0)]
G.add_weighted_edges_from(edge_list)

colors = ["red" for i in range(n)]


def draw_graph(G, colors):
    """Draws the graph with the chose colors"""
    layout = nx.shell_layout(G)
    nx.draw_networkx(G, node_color=colors, pos=layout)
    edge_labels = nx.get_edge_attributes(G, "weight")
    nx.draw_networkx_edge_labels(G, pos=layout, edge_labels=edge_labels)


draw_graph(G, colors)

すべてのエッジの重みに快適にアクセスするには、隣接行列を使用できます。この行列は[networkx.to_numpy_array()](https://networkx.org/documentation/stable/reference/generated/networkx.convert_matrix.to_numpy_array.html)で取得できます。

In [ ]:
w = nx.to_numpy_array(G)
print(w)

[`qiskit_optimization`](https://qiskit.org/documentation/optimization/index.html)モジュールには、隣接行列 $w$ から加重 Max-Cut 問題を定義できる[`Maxcut`](https://qiskit.org/documentation/optimization/stubs/qiskit_optimization.applications.Maxcut.html)というクラスが含まれています。その問題から、前のセクションで[`Maxcut.to_quadratic_program()`](https://qiskit.org/documentation/optimization/stubs/qiskit_optimization.applications.Maxcut.to_quadratic_program.html)を使用して推定した等価な２値最適化問題を得ることができます。

In [ ]:
from qiskit_optimization.applications import Maxcut

max_cut = Maxcut(w)

quadratic_program = max_cut.to_quadratic_program()
print(quadratic_program.prettyprint())

この問題は、2値最適化問題として表現できます。 $n$ をグラフのノード数とし、 $0 \leq i < n$ である各ノード（この場合 $n=4$ ）に対して、バイナリー変数 $x_i$ を考えます。この変数は、ノード $i$ が $1$ のラベルを貼ったグループの1つであれば値 $1$ を持ち、 $0$ とラベルを貼った他のグループの1つであれば値 $0$ を持つことになります。また、ノード $i$ からノード $j$ に至るエッジの重みを $w_{ij}$ （隣接行列 $w$ の要素 $(i,j)$ ）とします。グラフは無向性なので、 $w_{ij}=w_{ji}$ となります。そうすると、問題は以下のコスト関数を最大化するものとして定式化できます：

$$
\begin{aligned}
C(\vec{x})
&amp; =\sum_{i,j=0}^n w_{ij} x_i(1-x_j)\\[1mm]

&amp; = \sum_{i,j=0}^n w_{ij} x_i - \sum_{i,j=0}^n w_{ij} x_ix_j\\[1mm]

&amp; = \sum_{i,j=0}^n w_{ij} x_i - \sum_{i=0}^n \sum_{j=0}^i 2w_{ij} x_ix_j
\end{aligned}
$$

この問題を量子コンピューターで解くために、コスト関数を観測可能量の期待値として表現します。ただし、Qiskit が自然に認めている観測可能量は、 $0$ と $1$ の代わりに固有値 $1$ と $-1$ を持つパウリ演算子で構成されています。そのため、変数を次のように変更します。

ここで、 $\vec{x}=(x_0,x_1,\cdots ,x_{n-1})$ です。隣接行列 $w$ を使って、すべてのエッジの重みに快適にアクセスすることができます。これはコスト関数を求めるのに使われます：

$$
z_i = 1-2x_i \rightarrow x_i = \frac{1-z_i}{2}
$$

これは、次のことを意味します。

$$
\begin{array}{lcl} x_i=0 & \rightarrow & z_i=1 \\ x_i=1 & \rightarrow & z_i=-1.\end{array}
$$

したがって、最大化したい新しいコスト関数は次のとおりです。

$$
\begin{aligned}
C(\vec{z})
& = \sum_{i,j=0}^n w_{ij} \bigg(\frac{1-z_i}{2}\bigg)\bigg(1-\frac{1-z_j}{2}\bigg)\\[1mm]

& = \sum_{i,j=0}^n \frac{w_{ij}}{4} - \sum_{i,j=0}^n \frac{w_{ij}}{4} z_iz_j\\[1mm]

& = \sum_{i=0}^n \sum_{j=0}^i \frac{w_{ij}}{2} -  \sum_{i=0}^n \sum_{j=0}^i \frac{w_{ij}}{2} z_iz_j
\end{aligned}
$$

さらに、量子コンピューターの自然な傾向として、最大値ではなく最小値（通常は最低エネルギー）を見つけるので、 $C(\vec{z})$ を最大化するのではなく、最小化します：

$$
-C(\vec{z}) =  \sum_{i=0}^n \sum_{j=0}^i \frac{w_{ij}}{2} z_iz_j -  \sum_{i=0}^n \sum_{j=0}^i \frac{w_{ij}}{2}
$$

$-1$ と $1$ の値を持つ変数を最小化するためのコスト関数ができたので、次のようにパウリの $Z$ との類推を行うことができます。

$$
z_i \equiv Z_i = \overbrace{I}^{n-1}\otimes ... \otimes \overbrace{Z}^{i} \otimes ... \otimes \overbrace{I}^{0}
$$

つまり、変数 $z_i$ は、量子ビット $i$ に作用する $Z$ ゲートと同等になります。さらに：

$$
Z_i|x_{n-1}\cdots x_0\rangle = z_i|x_{n-1}\cdots x_0\rangle \rightarrow \langle x_{n-1}\cdots x_0 |Z_i|x_{n-1}\cdots x_0\rangle = z_i
$$

次に、検討する観測可能量は次のとおりです。

$$
\hat{H} = \sum_{i=0}^n \sum_{j=0}^i \frac{w_{ij}}{2} Z_iZ_j
$$

後で独立項を追加する必要があります。

$$
\texttt{offset} = - \sum_{i=0}^n \sum_{j=0}^i \frac{w_{ij}}{2}
$$

この変換は、[`QuadraticProgram.to_ising()`](https://qiskit.org/documentation/optimization/stubs/qiskit_optimization.QuadraticProgram.to_ising.html)で行うことができます。

In [ ]:
observable, offset = quadratic_program.to_ising()
print("Offset:", offset)
print("Ising Hamiltonian:")
print(str(observable))

VQE インスタンスを使用して、次のように最適なパラメーターを見つけることができます。

In [ ]:
from qiskit.primitives import Estimator
from qiskit.algorithms.minimum_eigensolvers import VQE
from qiskit.algorithms.optimizers import COBYLA
from qiskit.circuit.library import TwoLocal
import numpy as np

ansatz = TwoLocal(observable.num_qubits, "rx", reps=1)
optimizer = COBYLA()

vqe = VQE(
    estimator=Estimator(),
    ansatz=ansatz,
    optimizer=optimizer,
    initial_point=np.zeros(ansatz.num_parameters),
)

result = vqe.compute_minimum_eigenvalue(observable)

print(result)

In [ ]:
from qiskit.quantum_info import Statevector

optimal_circuit = result.optimal_circuit.bind_parameters(result.optimal_parameters)

x = max_cut.sample_most_likely(Statevector(optimal_circuit))
print("energy:", result.eigenvalue.real)
print("time:", result.optimizer_time)
print("max-cut objective:", result.eigenvalue.real + offset)
print("solution:", x)
print("solution objective:", quadratic_program.objective.evaluate(x))

# plot results
colors = ["red" if x[i] == 0 else "orange" for i in range(n)]
draw_graph(G, colors)

このレッスンでは、次のことを学びました。

- カスタム変分アルゴリズムの書き方
- 変分アルゴリズムを適用して最小固有値を見つける方法
- 変分アルゴリズムを利用してアプリケーションのユースケースを解く方法

最終レッスンに進み、評価を受けてバッジを獲得しましょう!